# Galaxy Branching Workflow - Model Variations Included

This notebook has two branches and four clustering families so the report and code match.

**Branches**
- A. Full-sample branch (photometry only)
- B. Emission-line branch (BPT subset with S/N > 3 on all four lines)

**Model variations on the photometry branch**
- KMeans for k in {2,3,4,5,6}
- Gaussian Mixture Models (GMM, covariance="full") for components in {2,3,4,5,6}
- Agglomerative clustering with linkages {"ward","complete"} and clusters in {2,3,4,5,6}
- DBSCAN with an eps heuristic from the k-distance curve


In [ ]:
# Setup
import os, json, warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score
from sklearn.neighbors import NearestNeighbors, kneighbors_graph, KNeighborsClassifier
from scipy.stats import chi2_contingency

plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['axes.grid'] = False

pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 200)

FIGDIR = Path('./figs')
FIGDIR.mkdir(parents=True, exist_ok=True)
print('Libraries imported.')

## 0. Load data
Set `DATA_PATH` to a CSV.
The following columns should exist.
- **Photometry / geometry:** `dered_u, dered_g, dered_r, dered_i, dered_z, petroR90_r, petroR50_r, modelMag_r`
- **BPT lines:** `h_beta_flux, oiii_5007_flux, h_alpha_flux, nii_6584_flux` and corresponding `_err` columns.


In [ ]:
OUT_DIR = Path('out')
OUT_DIR.mkdir(exist_ok=True)

# --- Deterministic loader: either ONE wide file or TWO separate files ---
DATA_DIR = Path('.')
GAL_PATH = DATA_DIR / 'galaxies_100k.csv'
EM_PATH = DATA_DIR / 'emission_lines_full.csv'

BPT_COLS = [
    "h_beta_flux","oiii_5007_flux","h_alpha_flux","nii_6584_flux",
    "h_beta_flux_err","oiii_5007_flux_err","h_alpha_flux_err","nii_6584_flux_err"
]

def _read_any(path):
    if path.suffix == ".parquet":
        return pd.read_parquet(path)
    if path.suffix == ".csv":
        return pd.read_csv(path)
    raise ValueError("Use CSV or Parquet")

if EM_PATH:
    # Two-file mode: load and merge
    df_phot = _read_any(GAL_PATH)
    df_spec = _read_any(EM_PATH)
    # Choose a key present in both
    key = "specObjID" if ("specObjID" in df_phot.columns and "specObjID" in df_spec.columns) else "objID"
    if key not in df_phot or key not in df_spec:
        raise RuntimeError("Cannot find a common key (specObjID or objID) in both files")

    # One spectrum per object: keep the one with best minimum S/N across the 4 lines
    for line, err in [("h_alpha_flux","h_alpha_flux_err"),
                      ("h_beta_flux","h_beta_flux_err"),
                      ("oiii_5007_flux","oiii_5007_flux_err"),
                      ("nii_6584_flux","nii_6584_flux_err")]:
        if line in df_spec and err in df_spec:
            df_spec[f"SN_{line}"] = df_spec[line] / df_spec[err]
    sn_cols = [c for c in df_spec.columns if c.startswith("SN_")]
    df_spec["SN_min"] = df_spec[sn_cols].min(axis=1) if sn_cols else 0.0

    spec_best = (df_spec
                 .sort_values("SN_min", ascending=False)
                 .drop_duplicates(subset=[key], keep="first"))

    # Merge left to preserve the full-sample branch
    keep_cols = [key] + [c for c in BPT_COLS if c in spec_best.columns]
    df = df_phot.merge(spec_best[keep_cols], on=key, how="left", validate="one_to_one")
else:
    # One-file mode: wide file already
    df = _read_any(GAL_PATH)

# Diagnostics
have_bpt = [c for c in BPT_COLS if c in df.columns]
print("Rows:", len(df), "| BPT columns present:", len(have_bpt), "/", len(BPT_COLS))
if len(have_bpt) and len(have_bpt) < 8:
    print("Partial BPT block found. You can still run the full-sample branch; BPT branch will be limited.")


In [ ]:
# Load with safer inference and consistent join key
galaxies = pd.read_csv(GAL_PATH, low_memory=False, dtype={'specObjID': str})
emlines  = pd.read_csv(EM_PATH,  low_memory=False, dtype={'specObjID': str})

galaxies['specObjID'] = galaxies['specObjID'].astype(str)
emlines['specObjID']  = emlines['specObjID'].astype(str)

# Coerce all non-key cols to numeric (invalid -> NaN) and log
def coerce_numeric(df, key_cols=('specObjID',)):
    coerced = []
    for c in df.columns:
        if c in key_cols: 
            continue
        before_num = pd.api.types.is_numeric_dtype(df[c])
        df[c] = pd.to_numeric(df[c], errors='coerce')
        if not before_num:
            coerced.append(c)
    return coerced

coerced_g = coerce_numeric(galaxies)
coerced_e = coerce_numeric(emlines)

with open(OUT_DIR/'coercion_log.json','w') as f:
    json.dump({'coerced_galaxies': coerced_g, 'coerced_emlines': coerced_e}, f, indent=2)

df = galaxies.merge(emlines, on='specObjID', how='left')
print('Merged shape:', df.shape)

## 1. Light feature engineering (shared between both branches)
- Colors: u-g, g-r, r-i, i-z
- Concentration index: R90/R50
- Surface brightness proxy: mu_r_proxy = modelMag_r + 2.5 log10(2π R50^2)

In [ ]:
df_fe = df.copy()
for a,b in [('u','g'), ('g','r'), ('r','i'), ('i','z')]:
    ca, cb = f'dered_{a}', f'dered_{b}'
    if ca in df_fe.columns and cb in df_fe.columns:
        df_fe[f'color_{a}_minus_{b}'] = df_fe[ca] - df_fe[cb]
if {'petroR90_r','petroR50_r'}.issubset(df_fe.columns):
    df_fe['concentration_r'] = df_fe['petroR90_r'] / df_fe['petroR50_r']
if {'modelMag_r','petroR50_r'}.issubset(df_fe.columns):
    r50 = df_fe['petroR50_r'].replace({0: np.nan})
    df_fe['mu_r_proxy'] = df_fe['modelMag_r'] + 2.5*np.log10(2*np.pi*(r50**2))
print('Feature engineering complete. Shape:', df_fe.shape)

## 2. BPT columns and missingness pattern
We treat the eight BPT columns as non-imputable and use them only in the emission-line branch.

In [ ]:
present_bpt = [c for c in BPT_COLS if c in df_fe.columns]
missing_block_mask = df_fe[present_bpt].isna().all(axis=1) if present_bpt else pd.Series(False, index=df_fe.index)
print('BPT columns present:', present_bpt)
print('Rows missing all present BPT columns:', int(missing_block_mask.sum()), '/', len(df_fe))

## 3. Utilities

In [ ]:
def build_feature_matrix(df_in, include_cols, exclude_cols=None, impute=True, scale=True):
    exclude_cols = set(exclude_cols or [])
    cols = [c for c in include_cols if c in df_in.columns]
    impute_cols = [c for c in cols if c not in exclude_cols]
    X_parts, used_cols = [], []
    if impute_cols:
        X_imp = df_in[impute_cols].to_numpy(dtype=np.float32)
        if impute:
            X_imp = SimpleImputer(strategy='median').fit_transform(X_imp).astype(np.float32)
        # Drop near-constant features to avoid zero-IQR scaling
        q75 = np.nanpercentile(X_imp, 75, axis=0)
        q25 = np.nanpercentile(X_imp, 25, axis=0)
        iqr = q75 - q25
        keep_idx = np.where(iqr > 1e-6)[0]
        if len(keep_idx) < X_imp.shape[1]:
            print('Dropping near-constant features:', int(X_imp.shape[1]-len(keep_idx)))
        X_imp = X_imp[:, keep_idx]
        used_cols = [impute_cols[i] for i in keep_idx]
        if scale:
            X_imp = RobustScaler().fit_transform(X_imp).astype(np.float32)
        X_parts.append(X_imp)
    X = np.concatenate(X_parts, axis=1) if X_parts else np.empty((len(df_in), 0), dtype=np.float32)
    return X, used_cols

def kewley_2001(x):
    return 0.61/(x - 0.47) + 1.19
def kauffmann_2003(x):
    return 0.61/(x - 0.05) + 1.30
def plot_bpt_annotated(df_bpt, title='BPT (S/N>3 on all lines)'):
    x = np.log10(df_bpt['nii_6584_flux'] / df_bpt['h_alpha_flux'])
    y = np.log10(df_bpt['oiii_5007_flux'] / df_bpt['h_beta_flux'])
    fig, ax = plt.subplots(figsize=(8,6))
    hb = ax.hexbin(x, y, gridsize=70, mincnt=1)
    fig.colorbar(hb, ax=ax, label='Counts/bin')
    xx = np.linspace(-1.6, 0.6, 1000)
    mask_kew = xx < (0.47 - 1e-6)
    mask_kau = xx < (0.05 - 1e-6)
    ax.plot(xx[mask_kew], kewley_2001(xx[mask_kew]), '--', label='Kewley 2001')
    ax.plot(xx[mask_kau], kauffmann_2003(xx[mask_kau]), ':',  label='Kauffmann 2003')
    ax.set_xlim(-1.6, 0.6); ax.set_ylim(-1.2, 1.5)
    ax.set_xlabel('log10([NII] 6584 / Halpha)')
    ax.set_ylabel('log10([OIII] 5007 / Hbeta)')
    ax.set_title(title); ax.legend(loc='lower right')
    ax.text(-1.25, -0.75, 'Star-forming', fontsize=10, color='white',
            bbox=dict(facecolor='black', alpha=0.5, boxstyle='round,pad=0.2'))
    ax.text(-0.55,  0.45, 'Composite',    fontsize=10, color='white',
            bbox=dict(facecolor='black', alpha=0.5, boxstyle='round,pad=0.2'))
    ax.text( 0.10,  0.85, 'AGN',          fontsize=10, color='white',
            bbox=dict(facecolor='black', alpha=0.5, boxstyle='round,pad=0.2'))
    plt.tight_layout(); plt.show()

---
# A. Full-sample branch - Photometry only

In [ ]:
id_like = {'specObjID','objID','flags','type','mode'}
photometric_like = [c for c in df_fe.columns if c.startswith('dered_') or c in {
    'modelMag_r','petroR90_r','petroR50_r','concentration_r','mu_r_proxy'
} or c.startswith('color_')]
full_cols = [c for c in photometric_like if c not in id_like and pd.api.types.is_numeric_dtype(df_fe[c])]
X_full, cols_full = build_feature_matrix(df_fe, include_cols=full_cols, exclude_cols=None, impute=True, scale=True)
print('Full branch matrix:', X_full.shape, '| features:', len(cols_full))

In [ ]:
pca_full = PCA(n_components=10, random_state=0).fit(X_full)
plt.plot(np.cumsum(pca_full.explained_variance_ratio_), marker='o')
plt.xlabel('Components'); plt.ylabel('Cumulative explained variance')
plt.title('Full-sample PCA variance curve')
plt.tight_layout(); plt.savefig(FIGDIR/'fig_pca_variance.png', dpi=175); plt.show()

In [ ]:
# 1) Inspect what went into PCA
print("X_full shape:", X_full.shape)
print("Any inf/NaN? ", ~np.isfinite(X_full).all())

# 2) Find blown-up rows
row_norm = np.linalg.norm(X_full, axis=1)
print("Top 5 row norms:", np.sort(row_norm)[-5:])
bad_rows = np.where(row_norm > np.percentile(row_norm, 99.9))[0]
print("Potential outlier rows:", bad_rows[:10])

# 3) Check which features cause it (near-zero IQR after imputation)
from sklearn.impute import SimpleImputer
X_imp = SimpleImputer(strategy='median').fit_transform(df_fe[full_cols].to_numpy(dtype=np.float64))
q75 = np.nanpercentile(X_imp, 75, axis=0)
q25 = np.nanpercentile(X_imp, 25, axis=0)
iqr = q75 - q25
small_iqr_idx = np.where(iqr < 1e-6)[0]
small_iqr_cols = [full_cols[i] for i in small_iqr_idx]
print("Near-constant (tiny IQR) features:", small_iqr_cols[:20], f"(total {len(small_iqr_cols)})")


In [ ]:
# Drop near-constant features before scaling
keep = [c for i,c in enumerate(full_cols) if i not in set(small_iqr_idx)]
X_imp = SimpleImputer(strategy='median').fit_transform(df_fe[keep].to_numpy(dtype=np.float32))
X_full = RobustScaler().fit_transform(X_imp).astype(np.float32)

# Winsorize extreme rows to be safe
row_norm = np.linalg.norm(X_full, axis=1)
cap = np.percentile(row_norm, 99.9)
X_full[row_norm > cap] *= (cap / row_norm[row_norm > cap])[:, None]

km_full = KMeans(n_clusters=3, n_init=10, random_state=0)
labels_full_k3 = km_full.fit_predict(X_full)
Z2 = PCA(n_components=2, random_state=0).fit_transform(X_full)
plt.scatter(Z2[:,0], Z2[:,1], s=6, c=labels_full_k3, alpha=0.65)
plt.title('Full-sample: PCA(2) + KMeans(k=3)')
plt.xlabel('PC1'); plt.ylabel('PC2')
plt.tight_layout(); plt.savefig(FIGDIR/'fig_pca_kmeans.png', dpi=175); plt.show()

---
# C. Model variations on the photometry branch

In [ ]:
def sweep_gmm_robust(X, comps=(2,3,4,5,6),
                     cov_types=('full','diag'),
                     reg_list=(1e-6, 1e-5, 1e-4),
                     seed=0, max_iter=500, n_init=3):
    X = np.asarray(X, dtype=np.float64)  # <- critical
    rows = []
    for cov in cov_types:
        for k in comps:
            fitted = False
            for reg in reg_list:
                try:
                    gmm = GaussianMixture(
                        n_components=k,
                        covariance_type=cov,
                        reg_covar=reg,
                        init_params='kmeans',
                        n_init=n_init,
                        random_state=seed,
                        max_iter=max_iter,
                        tol=1e-3
                    )
                    labels = gmm.fit_predict(X)
                    sil = silhouette_score(X, labels) if len(set(labels)) > 1 else np.nan
                    rows.append({
                        "model": "GMM",
                        "covariance": cov,
                        "k": k,
                        "reg_covar": reg,
                        "silhouette": sil,
                        "BIC": gmm.bic(X),
                        "AIC": gmm.aic(X),
                        "status": "ok"
                    })
                    fitted = True
                    break  # stop escalating reg once it worked
                except ValueError as e:
                    rows.append({
                        "model": "GMM",
                        "covariance": cov,
                        "k": k,
                        "reg_covar": reg,
                        "silhouette": np.nan,
                        "BIC": np.nan,
                        "AIC": np.nan,
                        "status": f"fail: {str(e)[:80]}"
                    })
            if not fitted:
                print(f"[GMM] all reg_covar attempts failed for k={k}, cov={cov}")
    return pd.DataFrame(rows)


In [ ]:
def sweep_kmeans(X, k_list=(2,3,4,5,6), seed=0):
    rows=[]
    for k in k_list:
        print("Running kmeans for ", k)
        km = KMeans(n_clusters=k, n_init=10, random_state=seed)
        lab = km.fit_predict(X)
        sil = silhouette_score(X, lab) if len(set(lab))>1 else np.nan
        rows.append({'model':'KMeans','k':k,'silhouette':sil})
    return pd.DataFrame(rows)

def sweep_gmm(X, comps=(2,3,4,5,6), cov='full', seed=0):
    rows=[]
    for k in comps:
        gmm = GaussianMixture(n_components=k, covariance_type=cov, random_state=seed)
        lab = gmm.fit_predict(X)
        sil = silhouette_score(X, lab) if len(set(lab))>1 else np.nan
        rows.append({'model':f'GMM-{cov}','k':k,'silhouette':sil,'BIC':gmm.bic(X),'AIC':gmm.aic(X)})
    return pd.DataFrame(rows)



In [ ]:
def sweep_agglom_knn(X, k_list=(2,3,4,5,6), linkage='complete',
                     n_neighbors=25, seed=0, max_points=60000):
    """
    Agglomerative with sparse kNN connectivity.
    If X is huge, subsample to max_points, then optionally propagate labels.
    """
    X = np.asarray(X, dtype=np.float32)
    n = X.shape[0]

    subsample_idx = None
    X_work = X
    if n > max_points:
        rng = np.random.default_rng(seed)
        subsample_idx = rng.choice(n, size=max_points, replace=False)
        X_work = X[subsample_idx]

    conn = kneighbors_graph(X_work, n_neighbors=n_neighbors,
                            mode='distance', include_self=False, n_jobs=-1)

    rows, labels_out = [], {}
    for k in k_list:
        agg = AgglomerativeClustering(n_clusters=k, linkage=linkage, connectivity=conn)
        lab_sub = agg.fit_predict(X_work)
        sil = silhouette_score(X_work, lab_sub) if len(set(lab_sub)) > 1 else np.nan
        rows.append({"model": f"Agglomerative-{linkage}",
                     "k": k, "silhouette": sil,
                     "n_used": X_work.shape[0],
                     "neighbors": n_neighbors,
                     "subsample": subsample_idx is not None})
        labels_out[k] = (lab_sub, subsample_idx)
    return pd.DataFrame(rows), labels_out

def propagate_labels_knn(X_all, label_sub, subsample_idx, n_neighbors=7):
    knn = KNeighborsClassifier(n_neighbors=n_neighbors, weights='distance')
    knn.fit(X_all[subsample_idx], label_sub)
    return knn.predict(X_all)

In [ ]:
print("Running kmeans")
km_df  = sweep_kmeans(X_full)

print("Running gmm")
gmm_df = sweep_gmm_robust(X_full, cov_types=('full','diag'))

print("Running Agglomerative-complete with kNN connectivity…")
agg_c_df, agg_labels = sweep_agglom_knn(X_full, linkage='complete', n_neighbors=25, max_points=60000)
display(agg_c_df.sort_values('silhouette', ascending=False).round(3))

best = agg_c_df.sort_values('silhouette', ascending=False).iloc[0]
k_best_c = int(best['k'])
lab_sub, subs_idx = agg_labels[k_best_c]
labels_full_agg_c = (propagate_labels_knn(X_full, lab_sub, subs_idx, n_neighbors=7)
                     if subs_idx is not None else lab_sub)

print("Summarizing")
summary = pd.concat([km_df, gmm_df, agg_c_df], ignore_index=True)
display(summary.sort_values(['model','k']).round(3))
summary.to_csv(FIGDIR/'model_sweep_summary.csv', index=False)

In [ ]:
Z2_full = PCA(n_components=2, random_state=0).fit_transform(X_full)

def best_row(df, prefix):
    dfm = df[df["model"].str.startswith(prefix)]
    return None if dfm.empty else dfm.sort_values("silhouette", ascending=False).head(1).iloc[0]

# KMeans
row = best_row(summary, "KMeans")
if row is not None:
    k = int(row["k"])
    lab = KMeans(n_clusters=k, n_init=10, random_state=0).fit_predict(X_full)
    plt.scatter(Z2_full[:,0], Z2_full[:,1], s=6, c=lab, alpha=0.65)
    plt.title(f"KMeans best (k={k})"); plt.xlabel("PC1"); plt.ylabel("PC2")
    plt.tight_layout(); plt.savefig(FIGDIR/'fig_kmeans_best.png', dpi=175); plt.show()

# GMM
row = best_row(summary, "GMM")
if row is not None and pd.notna(row["silhouette"]):
    k = int(row["k"]); cov = row["covariance"]; reg = float(row["reg_covar"])
    gmm = GaussianMixture(n_components=k, covariance_type=cov, reg_covar=reg,
                          init_params='kmeans', n_init=3, random_state=0, max_iter=500)
    lab = gmm.fit_predict(np.asarray(X_full, dtype=np.float64))
    plt.scatter(Z2_full[:,0], Z2_full[:,1], s=6, c=lab, alpha=0.65)
    plt.title(f"GMM best (k={k}, cov={cov}, reg={reg:g})"); plt.xlabel("PC1"); plt.ylabel("PC2")
    plt.tight_layout(); plt.savefig(FIGDIR/'fig_gmm_best.png', dpi=175); plt.show()

# Agglomerative-complete (use the labels we just built)
plt.scatter(Z2_full[:,0], Z2_full[:,1], s=6, c=labels_full_agg_c, alpha=0.65)
plt.title(f"Agglomerative-complete best (k={k_best_c})"); plt.xlabel("PC1"); plt.ylabel("PC2")
plt.tight_layout(); plt.savefig(FIGDIR/'fig_agglomerative_complete_best.png', dpi=175); plt.show()


In [ ]:
k = 10
nbrs = NearestNeighbors(n_neighbors=k).fit(X_full)
distances, _ = nbrs.kneighbors(X_full)
kdist = np.sort(distances[:,k-1])
plt.plot(kdist)
plt.title('DBSCAN k-distance curve (k=10)')
plt.xlabel('Points sorted by distance to k-th neighbor')
plt.ylabel('Distance')
plt.tight_layout(); plt.savefig(FIGDIR/'fig_dbscan_kdist.png', dpi=175); plt.show()

eps = float(np.percentile(kdist, 95))
db = DBSCAN(eps=eps, min_samples=k).fit(X_full)
labels_db = db.labels_
n_noise = int(np.sum(labels_db == -1))
n_clusters_db = len(set(labels_db)) - (1 if -1 in labels_db else 0)
print(f'DBSCAN -> eps={eps:.3f}, min_samples={k}, clusters={n_clusters_db}, noise={n_noise}')

mask_core = labels_db != -1
sil_db = np.nan
if len(set(labels_db[mask_core])) > 1:
    sil_db = silhouette_score(X_full[mask_core], labels_db[mask_core])
print('DBSCAN silhouette (core only):', None if np.isnan(sil_db) else round(sil_db, 3))

Z2 = PCA(n_components=2, random_state=0).fit_transform(X_full)
plt.scatter(Z2[mask_core,0], Z2[mask_core,1], s=6, c=labels_db[mask_core], alpha=0.65)
plt.title('DBSCAN clusters (core only) in PCA(2)')
plt.xlabel('PC1'); plt.ylabel('PC2')
plt.tight_layout(); plt.savefig(FIGDIR/'fig_dbscan_core.png', dpi=175); plt.show()

---
# B. Emission-line branch - BPT subset

In [ ]:
required = ['h_beta_flux','oiii_5007_flux','h_alpha_flux','nii_6584_flux',
            'h_beta_flux_err','oiii_5007_flux_err','h_alpha_flux_err','nii_6584_flux_err']
present = [c for c in required if c in df_fe.columns]
if len(present) < 8:
    print('Not all BPT columns are present - skipping BPT branch in this run.')
else:
    pos_mask = (df_fe['h_beta_flux']>0) & (df_fe['oiii_5007_flux']>0) & (df_fe['h_alpha_flux']>0) & (df_fe['nii_6584_flux']>0)
    sn_mask  = (df_fe['h_beta_flux']/df_fe['h_beta_flux_err']>3) & (df_fe['oiii_5007_flux']/df_fe['oiii_5007_flux_err']>3) & \
               (df_fe['h_alpha_flux']/df_fe['h_alpha_flux_err']>3) & (df_fe['nii_6584_flux']/df_fe['nii_6584_flux_err']>3)
    em_mask = pos_mask & sn_mask
    df_em = df_fe.loc[em_mask].copy()
    print('Emission-line subset size:', df_em.shape)
    # BPT with hexbin background
    x = np.log10(df_em['nii_6584_flux'] / df_em['h_alpha_flux'])
    y = np.log10(df_em['oiii_5007_flux'] / df_em['h_beta_flux'])
    fig, ax = plt.subplots(figsize=(7.5,5.5))
    hb = ax.hexbin(x, y, gridsize=70, bins='log', mincnt=1)
    xx = np.linspace(-1.6, 0.6, 1000)
    ax.plot(xx[xx<0.47-1e-6], kewley_2001(xx[xx<0.47-1e-6]), '--', label='Kewley 2001')
    ax.plot(xx[xx<0.05-1e-6], kauffmann_2003(xx[xx<0.05-1e-6]), ':',  label='Kauffmann 2003')
    ax.set_xlim(-1.6, 0.6); ax.set_ylim(-1.2, 1.5)
    ax.set_xlabel('log10([NII] 6584 / Halpha)'); ax.set_ylabel('log10([OIII] 5007 / Hbeta)')
    ax.set_title('BPT hexbin density')
    plt.tight_layout(); plt.savefig(FIGDIR/'fig_bpt_density.png', dpi=175); plt.show()

    # Map photometric clusters onto emission-line subset and color BPT by cluster
    labels_series = pd.Series(labels_full_k3, index=df_fe.index, name='cluster')
    df_em2 = df_em.join(labels_series, how='left')
    n2ha = np.log10(df_em2['nii_6584_flux']/df_em2['h_alpha_flux'])
    o3hb = np.log10(df_em2['oiii_5007_flux']/df_em2['h_beta_flux'])
    plt.figure(figsize=(7.5,5.5))
    plt.scatter(n2ha, o3hb, s=6, c=df_em2['cluster'], alpha=0.6)
    xx = np.linspace(-1.6, 0.6, 1000)
    plt.plot(xx[xx<0.47-1e-6], kewley_2001(xx[xx<0.47-1e-6]), '--')
    plt.plot(xx[xx<0.05-1e-6], kauffmann_2003(xx[xx<0.05-1e-6]), ':')
    plt.xlim(-1.6, 0.6); plt.ylim(-1.2, 1.5)
    plt.xlabel('log10([NII] 6584 / Halpha)'); plt.ylabel('log10([OIII] 5007 / Hbeta)')
    plt.title('BPT colored by photometric clusters')
    plt.tight_layout(); plt.savefig(FIGDIR/'fig_bpt_by_cluster.png', dpi=175); plt.show()

    # BPT class, crosstab, and chi-square
    def bpt_class(n2, o3):
        if o3 > kewley_2001(n2):
            return 'AGN'
        if o3 > kauffmann_2003(n2):
            return 'Composite'
        return 'Star-forming'
    df_em2 = df_em2.assign(
        n2ha = np.log10(df_em2['nii_6584_flux']/df_em2['h_alpha_flux']),
        o3hb = np.log10(df_em2['oiii_5007_flux']/df_em2['h_beta_flux'])
    )
    df_em2['BPT'] = [bpt_class(n, o) for n,o in zip(df_em2['n2ha'], df_em2['o3hb'])]
    xtab = pd.crosstab(df_em2['cluster'], df_em2['BPT'])
    print('Crosstab counts:\n', xtab)
    chi2, p, dof, _ = chi2_contingency(xtab)
    print(f'Chi-square={chi2:.1f}, dof={dof}, p={p:.2e}')
    xtab.to_csv(FIGDIR/'bpt_crosstab_counts.csv')